# OIKAN Regression Benchmark Tests

This notebook provides comprehensive benchmarking of OIKANRegressor against standard regression models:

1. Model Performance:
   - MSE, RMSE, MAE, R²
   - Training and prediction time
   - Model complexity (number of terms)

2. Dataset Coverage:
   - Synthetic (linear, nonlinear)
   - Real-world (diabetes, boston housing)
   - Custom physics-based

In [1]:
import warnings
warnings.filterwarnings('ignore')

!pip install -qU oikan

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcugraph-cu12 24.12.0 requires pylibraft-cu12==24.12.*, but you have pylibraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 24.12.0 requires rmm-cu12==24.12.*, but you have rmm-cu12 25.2.0 which is incompatible.


In [2]:
!pip freeze | grep oikan

oikan==0.0.2.5


In [3]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from time import time
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.datasets import make_regression, load_diabetes
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from oikan.model import OIKANRegressor

np.random.seed(42)

In [4]:
def generate_datasets():
    datasets = {}
    
    # 1. Linear synthetic data
    X_lin, y_lin = make_regression(n_samples=1000, n_features=10, noise=0.1,
                                   random_state=42)
    datasets['Linear Synthetic'] = (X_lin, y_lin)
    
    # 2. Nonlinear synthetic data
    X_nonlin = np.random.randn(1000, 10)
    y_nonlin = (np.sin(X_nonlin[:, 0]) + np.square(X_nonlin[:, 1]) + 
                np.exp(X_nonlin[:, 2]/5) + np.tanh(X_nonlin[:, 3]))
    datasets['Nonlinear Synthetic'] = (X_nonlin, y_nonlin)
    
    # 3. Diabetes dataset
    X_diabetes, y_diabetes = load_diabetes(return_X_y=True)
    datasets['Diabetes'] = (X_diabetes, y_diabetes)
    
    # 4. Physics-based synthetic
    t = np.linspace(0, 10, 1000)
    X_phys = np.column_stack([t, np.sin(t), np.cos(t), t**2, np.exp(-t/5)])
    y_phys = 2*np.sin(t) + 0.5*t**2 - 0.1*np.exp(-t/5) + np.random.normal(0, 0.1, size=1000)
    datasets['Physics'] = (X_phys, y_phys)
    
    return datasets

In [5]:
def benchmark_model(model, X, y, model_name='OIKAN'):
    results = {}
    
    # Scale features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.2, random_state=42
    )
    
    try:
        # Training time
        start_time = time()
        if model_name == 'OIKAN':
            model.fit(X_train, y_train, epochs=100, lr=0.01)
        else:
            model.fit(X_train, y_train)
        train_time = time() - start_time
        
        # Prediction time
        start_time = time()
        y_pred = model.predict(X_test)
        predict_time = time() - start_time
        
        # Metrics
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        
        # Cross-validation score
        cv_scores = cross_val_score(model, X_scaled, y, 
                                   cv=5, scoring='r2')
        
        results.update({
            'Model': model_name,
            'MSE': mse,
            'RMSE': rmse,
            'MAE': mae,
            'R2': r2,
            'CV R2 Mean': cv_scores.mean(),
            'CV R2 Std': cv_scores.std(),
            'Train Time': train_time,
            'Predict Time': predict_time
        })
        
        # OIKAN-specific metrics
        if model_name == 'OIKAN':
            # Get symbolic predictions
            symbolic_pred = model.symbolic_predict(X_test)
            symbolic_mse = mean_squared_error(y_test, symbolic_pred)
            symbolic_r2 = r2_score(y_test, symbolic_pred)
            
            # Get formula and count terms
            formula = model.get_symbolic_formula()
            n_terms = sum(len(f.split('+')) for f in formula)
            
            results.update({
                'Symbolic MSE': symbolic_mse,
                'Symbolic R2': symbolic_r2,
                'Formula Terms': n_terms,
                'Formula': formula
            })
            
    except Exception as e:
        print(f"Error benchmarking {model_name}: {str(e)}")
        results = {
            'Model': model_name,
            'Error': str(e)
        }
    
    return results

In [6]:
# Initialize models
models = {
    'OIKAN': OIKANRegressor(hidden_dims=[32, 16]),
    'Linear': LinearRegression(),
    'Ridge': Ridge(alpha=1.0),
    'RandomForest': RandomForestRegressor(n_estimators=100),
    'MLP': MLPRegressor(hidden_layer_sizes=(32, 16), max_iter=500)
}

# Get datasets
datasets = generate_datasets()
results_list = []

# Run benchmarks
for dataset_name, (X, y) in datasets.items():
    print(f"\nBenchmarking {dataset_name} dataset...")
    
    for model_name, model in models.items():
        print(f"Testing {model_name}...")
        res = benchmark_model(model, X, y, model_name)
        res['Dataset'] = dataset_name
        results_list.append(res)

# Convert results to DataFrame
results_df = pd.DataFrame(results_list)

# Display summary table
summary = results_df.pivot_table(
    index=['Dataset', 'Model'],
    values=['R2', 'MSE', 'Train Time', 'Predict Time'],
    aggfunc='mean'
).round(4)

print("\nBenchmark Results Summary:")
print("==========================")
print(summary)


Benchmarking Linear Synthetic dataset...
Testing OIKAN...
Epoch [10/100], Loss: 16276.2676
Epoch [20/100], Loss: 14477.1758
Epoch [30/100], Loss: 12288.1221
Epoch [40/100], Loss: 9300.8398
Epoch [50/100], Loss: 5956.7285
Epoch [60/100], Loss: 3800.5037
Epoch [70/100], Loss: 1999.6949
Epoch [80/100], Loss: 1296.0496
Epoch [90/100], Loss: 810.8312
Epoch [100/100], Loss: 644.4294
Epoch [10/100], Loss: 17017.0723
Epoch [20/100], Loss: 15643.4512
Epoch [30/100], Loss: 13924.2871
Epoch [40/100], Loss: 11237.7910
Epoch [50/100], Loss: 8471.7334
Epoch [60/100], Loss: 5336.6484
Epoch [70/100], Loss: 2837.8865
Epoch [80/100], Loss: 1817.5593
Epoch [90/100], Loss: 1143.1967
Epoch [100/100], Loss: 756.4972
Epoch [10/100], Loss: 16412.0566
Epoch [20/100], Loss: 14628.0449
Epoch [30/100], Loss: 11947.4043
Epoch [40/100], Loss: 9112.9668
Epoch [50/100], Loss: 5373.9199
Epoch [60/100], Loss: 3029.9766
Epoch [70/100], Loss: 1648.5940
Epoch [80/100], Loss: 1014.9962
Epoch [90/100], Loss: 841.1264
Epoch

In [7]:
# Feature importance analysis for OIKAN
oikan_model = models['OIKANRegressor']

for dataset_name, (X, y) in datasets.items():
    print(f'\nFeature Importance Analysis for {dataset_name}:')
    X_scaled = StandardScaler().fit_transform(X)
    oikan_model.fit(X_scaled, y)
    
    feature_scores = oikan_model.get_feature_scores()
    
    plt.figure(figsize=(10, 4))
    plt.bar(range(len(feature_scores)), feature_scores)
    plt.title(f'OIKAN Feature Importance - {dataset_name}')
    plt.xlabel('Feature Index')
    plt.ylabel('Importance Score')
    plt.show()

KeyError: 'OIKANRegressor'